### DATA PT14-extra - Proyecto final - Grupo 3
#### Control de calidad de los datos del archivo Google Maps review de los estados de California (CA), Nevada (NV) y Georgia (GA)

In [2]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Obtener la lista de archivos JSON en carpeta
archivos_json = glob.glob(os.path.join("maps/lugar", "*.json"))

In [4]:
# Se guardará archivos JSON en formato pickle (binario) el cual es más rápido para futuras consultas
for archivo in archivos_json:
    df = pd.read_json(archivo, lines=True)
    df.to_pickle(archivo.replace(".json", ".pkl"))  # Guardar como .pkl

In [6]:
# Carga los archivos Pickle generados y los combina en un solo DataFrame.
df_list = [pd.read_pickle(archivo.replace(".json", ".pkl")) for archivo in archivos_json] # convierte el nombre de cada archivo de .json a .pkl para cargarlo correctamente.
df_rev = pd.concat(df_list, ignore_index=True)

df_rev.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [7]:
df_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  300000 non-null  float64
 1   name     300000 non-null  object 
 2   time     300000 non-null  int64  
 3   rating   300000 non-null  int64  
 4   text     182220 non-null  object 
 5   pics     10186 non-null   object 
 6   resp     40625 non-null   object 
 7   gmap_id  300000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 18.3+ MB


In [8]:
# Eliminamos, por ahora, pics y resp. no usaremos esas columnas
df_rev = df_rev.drop(columns=['pics', 'resp'])

In [9]:
# # reviso si hay reviews duplicadas
df_dup = df_rev[df_rev.duplicated(subset=['user_id', 'gmap_id', 'time'], keep=False)]
df_dup = df_dup.sort_values(['user_id', 'gmap_id', 'time'])

print(df_dup)

             user_id                 name           time  rating  \
13055   1.000011e+20            Dan Serns  1630954898274       5   
26998   1.000011e+20            Dan Serns  1630954898274       5   
765     1.000028e+20        dylan gunawan  1431527018712       5   
14708   1.000028e+20        dylan gunawan  1431527018712       5   
6087    1.000040e+20  Adrien bliťz-weiner  1520471265834       5   
...              ...                  ...            ...     ...   
15261   1.184430e+20         Deborah Katz  1520869457147       4   
3704    1.184440e+20         Stella López  1624940588056       5   
17647   1.184440e+20         Stella López  1624940588056       5   
150325  1.184459e+20        Menito Garcia  1599873059501       5   
151214  1.184459e+20        Menito Garcia  1599873059501       5   

                                                     text  \
13055          Great people telling everyone about Jesus!   
26998          Great people telling everyone about Jesus!   


Hay 36759 reviews duplicadas por las columnas 'user_id', 'gmap_id', 'time' para el estado de NJ. Las elimino

In [10]:
# eliminar los duplicados manteniendo la primera instancia
df_rev_sindup = df_rev.drop_duplicates(subset=['user_id', 'gmap_id', 'time'], keep='first')

In [11]:
# rename column name to user_name
df_rev_sindup.rename(columns={'name': 'user_name'}, inplace=True)

In [12]:
# convertir columna time en int64 a datetime
df_rev_sindup['date'] = pd.to_datetime(df_rev_sindup['time'], unit='ms')

In [13]:
# Eliminamos la columan time
df_rev_sindup = df_rev_sindup.drop(columns=['time'])
df_rev_sindup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 281360 entries, 0 to 299999
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   user_id    281360 non-null  float64       
 1   user_name  281360 non-null  object        
 2   rating     281360 non-null  int64         
 3   text       169314 non-null  object        
 4   gmap_id    281360 non-null  object        
 5   date       281360 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 15.0+ MB


In [15]:
# guardo df_rev_sindup
df_rev_sindup.to_parquet("maps\lugar\df_rev_sindup.parquet" , engine='fastparquet')

In [ ]:
# leo el archivo de las pizzeriaspara hacer join con este
df = pd.read_parquet("maps\df_resto_sindup_verif_coord.parquet" , engine='fastparquet')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 24711 to 3496
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gmap_id             7 non-null      object 
 1   business_name       7 non-null      object 
 2   address             7 non-null      object 
 3   city                7 non-null      object 
 4   state               7 non-null      object 
 5   zip_code            7 non-null      object 
 6   latitude            7 non-null      float64
 7   longitude           7 non-null      float64
 8   avg_rating          7 non-null      float64
 9   num_of_reviews      7 non-null      int64  
 10  verified_latitude   7 non-null      float64
 11  verified_longitude  7 non-null      float64
 12  lat_diff            7 non-null      float64
 13  long_diff           7 non-null      float64
dtypes: float64(7), int64(1), object(6)
memory usage: 840.0+ bytes


In [18]:
# cuento la cantidad de pizzerias en cada estado / no puedo contar con una sola columna
df[['state', 'business_name' ]].groupby('state').count()

,business_name
state,
CA,5
GA,1
NV,1


Googlemaps tiene datos de 5 pizzerias en CA, 1 en GA y 1 pizzerias en NV

In [ ]:
# separo las pizzerias. Luego hago left join con las reviews y vuelvo a unir los dataframe
df_CA = df[df['state'] == 'CA']
df_CA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 24711 to 9310
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gmap_id             5 non-null      object 
 1   business_name       5 non-null      object 
 2   address             5 non-null      object 
 3   city                5 non-null      object 
 4   state               5 non-null      object 
 5   zip_code            5 non-null      object 
 6   latitude            5 non-null      float64
 7   longitude           5 non-null      float64
 8   avg_rating          5 non-null      float64
 9   num_of_reviews      5 non-null      int64  
 10  verified_latitude   5 non-null      float64
 11  verified_longitude  5 non-null      float64
 12  lat_diff            5 non-null      float64
 13  long_diff           5 non-null      float64
dtypes: float64(7), int64(1), object(6)
memory usage: 600.0+ bytes


In [20]:
df_GA = df[df['state'] == 'GA']
df_GA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 3496 to 3496
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gmap_id             1 non-null      object 
 1   business_name       1 non-null      object 
 2   address             1 non-null      object 
 3   city                1 non-null      object 
 4   state               1 non-null      object 
 5   zip_code            1 non-null      object 
 6   latitude            1 non-null      float64
 7   longitude           1 non-null      float64
 8   avg_rating          1 non-null      float64
 9   num_of_reviews      1 non-null      int64  
 10  verified_latitude   1 non-null      float64
 11  verified_longitude  1 non-null      float64
 12  lat_diff            1 non-null      float64
 13  long_diff           1 non-null      float64
dtypes: float64(7), int64(1), object(6)
memory usage: 120.0+ bytes


In [21]:
df_NV = df[df['state'] == 'NV']
df_NV.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 15388 to 15388
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gmap_id             1 non-null      object 
 1   business_name       1 non-null      object 
 2   address             1 non-null      object 
 3   city                1 non-null      object 
 4   state               1 non-null      object 
 5   zip_code            1 non-null      object 
 6   latitude            1 non-null      float64
 7   longitude           1 non-null      float64
 8   avg_rating          1 non-null      float64
 9   num_of_reviews      1 non-null      int64  
 10  verified_latitude   1 non-null      float64
 11  verified_longitude  1 non-null      float64
 12  lat_diff            1 non-null      float64
 13  long_diff           1 non-null      float64
dtypes: float64(7), int64(1), object(6)
memory usage: 120.0+ bytes


Uno el dataframe de datos de pizzerias en un left join con el de los datos de reviews para obtener todas las reviews de pizzerias y agregarle el nombre del local. Mas adelante hare los mismo para los locales. Y luego unire los dos dataframes

In [34]:
df_unido = df.merge(df_CA, how='left', on='gmap_id', suffixes=('', '_CA')) \
             .merge(df_NV, how='left', on='gmap_id', suffixes=('', '_NV')) \
             .merge(df_GA, how='left', on='gmap_id', suffixes=('', '_GA'))


In [45]:
df_unido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gmap_id                7 non-null      object 
 1   business_name          7 non-null      object 
 2   address                7 non-null      object 
 3   city                   7 non-null      object 
 4   state                  7 non-null      object 
 5   zip_code               7 non-null      object 
 6   latitude               7 non-null      float64
 7   longitude              7 non-null      float64
 8   avg_rating             7 non-null      float64
 9   num_of_reviews         7 non-null      int64  
 10  verified_latitude      7 non-null      float64
 11  verified_longitude     7 non-null      float64
 12  lat_diff               7 non-null      float64
 13  long_diff              7 non-null      float64
 14  business_name_CA       5 non-null      object 
 15  address_CA

In [46]:
df_unido.head()

,gmap_id,business_name,address,city,state,zip_code,latitude,longitude,avg_rating,num_of_reviews,...,state_GA,zip_code_GA,latitude_GA,longitude_GA,avg_rating_GA,num_of_reviews_GA,verified_latitude_GA,verified_longitude_GA,lat_diff_GA,long_diff_GA
0,0x809467352f2ce435:0x9d7881ae2a25c1b0,Piara Pizza,3131 N Maroa Ave,Fresno,CA,93704,36.779041,-119.800262,4.3,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x80c2971865b00631:0x1d360a4bfe47e631,Chit Chat Restaurant Cafe Lounge,7115 Van Nuys Blvd,Van Nuys,CA,91405,34.199821,-118.449029,4.8,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x80c2c7726b53b8e5:0xad288d11654f5e48,PIZZA ROW RECORDS,433 E 7th St,Los Angeles,CA,90014,34.041406,-118.247477,4.3,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x80c2c7d5279490eb:0x4374ad061c2c67de,Danny Boy's Famous Original Pizza,330 S Hope St,Los Angeles,CA,90071,34.052914,-118.252486,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0x80c8dd7678249c49:0x6f2556114030dded,Domino's Pizza,6540 E Lake Mead Blvd,Las Vegas,NV,89156,36.196051,-115.030257,3.7,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# cuento la cantidad de reseñas de pizzerias en cada estado / no puedo contar con una sola columna
df_unido[['state', 'gmap_id' ]].groupby('state').count()

,gmap_id
state,
CA,5
GA,1
NV,1


In [48]:
# guardo df_pizza_NJNY_rev
df_unido.to_parquet('maps/df_estados.parquet' , engine='fastparquet')